In [ ]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import Acc_shelter


# Importing AnimalShelter class from Acc_shelter
from Acc_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
username = "aaacuser"
password = "Lebronjames23!"
animals = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(animals.read({"animal_type":"Dog"}))





#########################
# Dashboard Layout / View
#########################
# create dash application

app = dash.Dash(__name__)


image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                               
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Elohim Jacksons Grazioso Salvare Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='water'
        )
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Allows 10 rows per page, and starts on page 0. Columns and Rows may be selected but not deleted/edited

        editable=True,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(rescue_type):
        # Water Rescue filter
        if rescue_type == 'water':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Mountain or Wilderness Rescue filter
        elif rescue_type == 'mountain':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Rescue or Individual Tracking filter
        elif rescue_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(animals.read({"animal_type":"Dog"}))
        
        # allow columns to be deletebale 
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    ])
def update_graphs(viewData):
    #Histogram
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='breed')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        dl.Map(style={'width': '800px', 'height': '400px'}, 
               center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], 
               zoom=10, 
               children=[
                   dl.TileLayer(id="base-layer-id"),
            #Tool tip and popup
                dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], 
                children=[
                # Parameters 0-4 gives breed
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # Parameter 0-9 gives animals name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Feb/2023 02:13:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:47] "GET /_favicon.ico?v=1.10.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:47] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [18/Feb/2023 02:13:48] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Feb/2023 02:13:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:48] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [18/Feb/2023 02:13:49] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Feb/2023 02:13:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:13:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:14:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:14:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:14:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:14:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:14:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 02:14:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.